<a href="https://colab.research.google.com/github/ayanbabusona/Data-Analysis/blob/master/carnival_wars.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np, os
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_df = pd.read_csv('/content/drive/My Drive/carnival-wars/dataset/train.csv')
train_df.head()

,Product_id,Stall_no,instock_date,Market_Category,Customer_name,Loyalty_customer,Product_Category,Grade,Demand,Discount_avail,charges_1,charges_2 (%),Minimum_price,Maximum_price,Selling_Price
0,BRAE2NF6JA5GUEXG,37.0,2015-08-22 18:36:12.000,2,Lillyann,Yes,Fashion,1,68,0.0,376.0,11.0,2983.0,4713.0,4185.947700
1,TUNE8SFB6RJN2HSD,38.0,2016-03-27 21:19:13.000,24,Klynn,Yes,Fashion,0,51,0.0,397.0,12.0,7495.0,10352.0,9271.490256
2,BRAEAR7WZPQGPBZU,9.0,2015-08-18 19:25:22.000,447,Ridge,Yes,Child_care,0,10,0.0,250.0,9.0,5752.0,7309.0,6785.701362
3,WATDZ2ZQ8JPDHCTJ,50.0,2016-03-28 21:53:01.000,23,Abran,Yes,Educational,2,48,0.0,144.0,13.0,5090.0,20814.0,13028.917824
4,JWSEBUKYQPMBZ3RK,7.0,2016-03-29 22:58:53.000,63,Dustyn,Yes,Repair,1,35,1.0,211.0,4.0,2430.0,9261.0,906.553935


In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6368 entries, 0 to 6367
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Product_id        6368 non-null   object 
 1   Stall_no          6363 non-null   float64
 2   instock_date      6368 non-null   object 
 3   Market_Category   6368 non-null   int64  
 4   Customer_name     6157 non-null   object 
 5   Loyalty_customer  6368 non-null   object 
 6   Product_Category  6368 non-null   object 
 7   Grade             6368 non-null   int64  
 8   Demand            6368 non-null   int64  
 9   Discount_avail    6330 non-null   float64
 10  charges_1         6170 non-null   float64
 11  charges_2 (%)     6163 non-null   float64
 12  Minimum_price     6330 non-null   float64
 13  Maximum_price     6025 non-null   float64
 14  Selling_Price     6327 non-null   float64
dtypes: float64(7), int64(3), object(5)
memory usage: 746.4+ KB


In [ ]:
train_df.dropna(inplace=True)
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5397 entries, 0 to 6367
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Product_id        5397 non-null   object 
 1   Stall_no          5397 non-null   float64
 2   instock_date      5397 non-null   object 
 3   Market_Category   5397 non-null   int64  
 4   Customer_name     5397 non-null   object 
 5   Loyalty_customer  5397 non-null   object 
 6   Product_Category  5397 non-null   object 
 7   Grade             5397 non-null   int64  
 8   Demand            5397 non-null   int64  
 9   Discount_avail    5397 non-null   float64
 10  charges_1         5397 non-null   float64
 11  charges_2 (%)     5397 non-null   float64
 12  Minimum_price     5397 non-null   float64
 13  Maximum_price     5397 non-null   float64
 14  Selling_Price     5397 non-null   float64
dtypes: float64(7), int64(3), object(5)
memory usage: 674.6+ KB


In [ ]:
test_df = pd.read_csv('/content/drive/My Drive/carnival-wars/dataset/test.csv')
test_df.head()

,Product_id,Stall_no,instock_date,Market_Category,Customer_name,Loyalty_customer,Product_Category,Grade,Demand,Discount_avail,charges_1,charges_2 (%),Minimum_price,Maximum_price
0,SCHE4YSTDVPVZVXW,39.0,2016-01-13 07:45:08.000,205,Ivanka,No,Fashion,0,90,0,380.0,16.0,2576.0,3340
1,ACCEGCATKHNRXUHW,49.0,2015-08-23 20:37:05.000,3,Isaak,Yes,Fashion,0,87,0,393.0,16.0,1202.0,1955
2,NKCE6GJ5XVJDXNNZ,1.0,2015-11-14 18:12:39.000,183,Analiese,No,Technology,2,55,0,493.0,9.0,7175.0,15715
3,NKCEB8BK3ZXDHDHM,8.0,2015-11-21 04:56:19.000,358,Rusty,Yes,Child_care,3,86,0,303.0,16.0,5404.0,13078
4,TOPEFDXSAHRNPF94,33.0,2015-05-06 15:09:46.000,167,Eloise,No,Technology,0,27,0,567.0,16.0,4069.0,6244


In [ ]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3430 entries, 0 to 3429
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Product_id        3430 non-null   object 
 1   Stall_no          3429 non-null   float64
 2   instock_date      3430 non-null   object 
 3   Market_Category   3430 non-null   int64  
 4   Customer_name     3377 non-null   object 
 5   Loyalty_customer  3430 non-null   object 
 6   Product_Category  3430 non-null   object 
 7   Grade             3430 non-null   int64  
 8   Demand            3430 non-null   int64  
 9   Discount_avail    3430 non-null   int64  
 10  charges_1         3394 non-null   float64
 11  charges_2 (%)     3425 non-null   float64
 12  Minimum_price     3416 non-null   float64
 13  Maximum_price     3430 non-null   int64  
dtypes: float64(4), int64(5), object(5)
memory usage: 375.3+ KB


In [ ]:
dataset = [train_df, test_df]
for data in dataset:
    data['instock_date'] = pd.to_datetime(data['instock_date'])
    data['Loyalty_customer'] = [1 if (Loyalty_customer == 'Yes') else 0  for Loyalty_customer in data.Loyalty_customer]

In [ ]:
test_df

,Product_id,Stall_no,instock_date,Market_Category,Customer_name,Loyalty_customer,Product_Category,Grade,Demand,Discount_avail,charges_1,charges_2 (%),Minimum_price,Maximum_price
0,SCHE4YSTDVPVZVXW,39.0,2016-01-13 07:45:08,205,Ivanka,0,Fashion,0,90,0,380.0,16.0,2576.0,3340
1,ACCEGCATKHNRXUHW,49.0,2015-08-23 20:37:05,3,Isaak,1,Fashion,0,87,0,393.0,16.0,1202.0,1955
2,NKCE6GJ5XVJDXNNZ,1.0,2015-11-14 18:12:39,183,Analiese,0,Technology,2,55,0,493.0,9.0,7175.0,15715
3,NKCEB8BK3ZXDHDHM,8.0,2015-11-21 04:56:19,358,Rusty,1,Child_care,3,86,0,303.0,16.0,5404.0,13078
4,TOPEFDXSAHRNPF94,33.0,2015-05-06 15:09:46,167,Eloise,0,Technology,0,27,0,567.0,16.0,4069.0,6244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3425,BBAE9K7BXFUTDNBK,9.0,2015-08-15 21:17:17,58,Davion,1,Pet_care,0,39,0,117.0,5.0,3094.0,4674
3426,RTRDYPA4PJHCJCGF,30.0,2015-12-04 01:51:47,358,Maiah,1,Technology,3,17,0,509.0,6.0,3384.0,14105
3427,TUNE8FWQBS7TVVT7,14.0,2016-03-28 20:55:47,106,Gwendolen,1,Educational,0,36,0,150.0,17.0,710.0,10894
3428,NKCEGMUABCRGUZPE,1.0,2015-05-23 07:41:36,104,Addilynn,1,Child_care,0,63,0,325.0,15.0,1477.0,5165


In [ ]:
for data in dataset:
  data['Product_Category'] = data['Product_Category'].replace(['Fashion'],1)
  data['Product_Category'] = data['Product_Category'].replace(['Technology'],2)
  data['Product_Category'] = data['Product_Category'].replace(['Child_care'],3)
  data['Product_Category'] = data['Product_Category'].replace(['Hospitality'],4)
  data['Product_Category'] = data['Product_Category'].replace(['Organic'],5)
  data['Product_Category'] = data['Product_Category'].replace(['Pet_care'],6)
  data['Product_Category'] = data['Product_Category'].replace(['Repair'],7)
  data['Product_Category'] = data['Product_Category'].replace(['Cosmetics'],8)
  data['Product_Category'] = data['Product_Category'].replace(['Home_decor'],9)
  data['Product_Category'] = data['Product_Category'].replace(['Educational'],10)

In [ ]:
test_df

,Product_id,Stall_no,instock_date,Market_Category,Customer_name,Loyalty_customer,Product_Category,Grade,Demand,Discount_avail,charges_1,charges_2 (%),Minimum_price,Maximum_price
0,SCHE4YSTDVPVZVXW,39.0,2016-01-13 07:45:08,205,Ivanka,0,1,0,90,0,380.0,16.0,2576.0,3340
1,ACCEGCATKHNRXUHW,49.0,2015-08-23 20:37:05,3,Isaak,1,1,0,87,0,393.0,16.0,1202.0,1955
2,NKCE6GJ5XVJDXNNZ,1.0,2015-11-14 18:12:39,183,Analiese,0,2,2,55,0,493.0,9.0,7175.0,15715
3,NKCEB8BK3ZXDHDHM,8.0,2015-11-21 04:56:19,358,Rusty,1,3,3,86,0,303.0,16.0,5404.0,13078
4,TOPEFDXSAHRNPF94,33.0,2015-05-06 15:09:46,167,Eloise,0,2,0,27,0,567.0,16.0,4069.0,6244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3425,BBAE9K7BXFUTDNBK,9.0,2015-08-15 21:17:17,58,Davion,1,6,0,39,0,117.0,5.0,3094.0,4674
3426,RTRDYPA4PJHCJCGF,30.0,2015-12-04 01:51:47,358,Maiah,1,2,3,17,0,509.0,6.0,3384.0,14105
3427,TUNE8FWQBS7TVVT7,14.0,2016-03-28 20:55:47,106,Gwendolen,1,10,0,36,0,150.0,17.0,710.0,10894
3428,NKCEGMUABCRGUZPE,1.0,2015-05-23 07:41:36,104,Addilynn,1,3,0,63,0,325.0,15.0,1477.0,5165


In [ ]:
train_df

,Product_id,Stall_no,instock_date,Market_Category,Customer_name,Loyalty_customer,Product_Category,Grade,Demand,Discount_avail,charges_1,charges_2 (%),Minimum_price,Maximum_price,Selling_Price
0,BRAE2NF6JA5GUEXG,37.0,2015-08-22 18:36:12,2,Lillyann,1,1,1,68,0.0,376.0,11.0,2983.0,4713.0,4185.947700
1,TUNE8SFB6RJN2HSD,38.0,2016-03-27 21:19:13,24,Klynn,1,1,0,51,0.0,397.0,12.0,7495.0,10352.0,9271.490256
2,BRAEAR7WZPQGPBZU,9.0,2015-08-18 19:25:22,447,Ridge,1,3,0,10,0.0,250.0,9.0,5752.0,7309.0,6785.701362
3,WATDZ2ZQ8JPDHCTJ,50.0,2016-03-28 21:53:01,23,Abran,1,10,2,48,0.0,144.0,13.0,5090.0,20814.0,13028.917824
4,JWSEBUKYQPMBZ3RK,7.0,2016-03-29 22:58:53,63,Dustyn,1,7,1,35,1.0,211.0,4.0,2430.0,9261.0,906.553935
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6363,WATE7VE2D62VEKW8,17.0,2016-03-28 19:55:43,24,Marquell,1,2,0,47,0.0,614.0,10.0,8174.0,11266.0,10277.520192
6364,SHOEF9G8Z4ZXF6WE,21.0,2015-02-11 15:09:26,55,Theodore,1,3,0,35,0.0,269.0,5.0,3564.0,4920.0,4501.837200
6365,PBXEB9ZQ7HHCFXYG,36.0,2015-11-19 17:29:21,358,Galen,0,7,3,29,0.0,283.0,3.0,5303.0,14614.0,10218.878775
6366,NKCEAGXHRYTNTMG7,27.0,2015-12-08 04:19:33,452,Posey,1,7,0,8,0.0,267.0,3.0,4334.0,5849.0,5359.493997


In [ ]:
# Selected features for training
features = ['Stall_no', 'Market_Category', 'Loyalty_customer', 'Product_Category', 'Grade', 'Demand',	'Discount_avail', 'charges_1', 'charges_2 (%)', 'Minimum_price', 'Maximum_price']

# Saving Normalizing Parameters for future use with train and test set
feature_min = train_df[features].min().values
feature_max = train_df[features].max().values

# Normalization 
train_df_norm = (train_df[features] - feature_min)/ (feature_max - feature_min)
X_test = (test_df[features] - feature_min)/ (feature_max - feature_min)

y = train_df['Selling_Price']
X_train, X_valid, y_train, y_valid = train_test_split( train_df_norm, y, test_size=0.2, random_state=1234)

In [ ]:
# LGBM Parameters
hyper_params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': ['l2', 'auc'],
    'learning_rate': 0.00005,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.7,
    'bagging_freq': 10,
    'verbose': 0,
    "max_depth": 8,
    "num_leaves": 128,  
    "max_bin": 512,
    "num_iterations": 100000,
    "n_estimators": 1000
}


gbm = lgb.LGBMRegressor(**hyper_params)

In [ ]:
gbm.fit(X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        eval_metric='auc',
        early_stopping_rounds=1000)

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's l2: 1.01222e+07	valid_0's auc: 0.499069
Training until validation scores don't improve for 1000 rounds.
[2]	valid_0's l2: 1.01212e+07	valid_0's auc: 0.499069
[3]	valid_0's l2: 1.01207e+07	valid_0's auc: 0.496508
[4]	valid_0's l2: 1.01198e+07	valid_0's auc: 0.494025
[5]	valid_0's l2: 1.01188e+07	valid_0's auc: 0.49325
[6]	valid_0's l2: 1.01183e+07	valid_0's auc: 0.496508
[7]	valid_0's l2: 1.01177e+07	valid_0's auc: 0.504578
[8]	valid_0's l2: 1.01169e+07	valid_0's auc: 0.505975
[9]	valid_0's l2: 1.01159e+07	valid_0's auc: 0.500698
[10]	valid_0's l2: 1.01153e+07	valid_0's auc: 0.502716
[11]	valid_0's l2: 1.01147e+07	valid_0's auc: 0.509466
[12]	valid_0's l2: 1.0114e+07	valid_0's auc: 0.514898
[13]	valid_0's l2: 1.01132e+07	valid_0's auc: 0.511173
[14]	valid_0's l2: 1.01127e+07	valid_0's auc: 0.518001
[15]	valid_0's l2: 1.0112e+07	valid_0's auc: 0.521415
[16]	valid_0's l2: 1.01114e+07	valid_0's auc: 0.52576
[17]	valid_0's l2: 1.01104e+07	valid_0's auc: 0.523122
[18]	valid_0's

LGBMRegressor(bagging_fraction=0.7, bagging_freq=10, boosting_type='gbdt',
              class_weight=None, colsample_bytree=1.0, feature_fraction=0.9,
              importance_type='split', learning_rate=5e-05, max_bin=512,
              max_depth=8, metric=['l2', 'auc'], min_child_samples=20,
              min_child_weight=0.001, min_split_gain=0.0, n_estimators=1000,
              n_jobs=-1, num_iterations=100000, num_leaves=128,
              objective='regression', random_state=None, reg_alpha=0.0,
              reg_lambda=0.0, silent=True, subsample=1.0,
              subsample_for_bin=200000, subsample_freq=0, task='train',
              verbose=0)

In [ ]:
y_test_predicted = gbm.predict(X_test, num_iteration=gbm.best_iteration_)

In [ ]:
submission = pd.DataFrame({'Product_id' : test_df['Product_id'], 'Selling_Price': y_test_predicted})
submission.to_csv('submission_auc_4.csv')

In [ ]:
import random 
FOLD = 20; 
for i in range(FOLD):
    X_train, X_valid, y_train, y_valid = train_test_split( train_df_norm, y, test_size=0.2, random_state=random.randint(10, 1000))

    # gbm = lgb.LGBMRegressor(**hyper_params)
    gbm.fit(X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        eval_metric='auc',
        early_stopping_rounds=1000)
    
    y_test_predicted = gbm.predict(X_test, num_iteration=gbm.best_iteration_)

    submission = pd.DataFrame({'Product_id' : test_df['Product_id'], 'Selling_Price': y_test_predicted})
    submission.to_csv(f'submission_auc_{i}.csv', index = False)

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Streaming output truncated to the last 5000 lines.
[128]	valid_0's l2: 3.05843e+06	valid_0's auc: 0.384508
[129]	valid_0's l2: 3.04577e+06	valid_0's auc: 0.385436
[130]	valid_0's l2: 3.0183e+06	valid_0's auc: 0.384508
[131]	valid_0's l2: 2.99126e+06	valid_0's auc: 0.384508
[132]	valid_0's l2: 2.96442e+06	valid_0's auc: 0.384972
[133]	valid_0's l2: 2.95137e+06	valid_0's auc: 0.384972
[134]	valid_0's l2: 2.92527e+06	valid_0's auc: 0.3859
[135]	valid_0's l2: 2.90303e+06	valid_0's auc: 0.3859
[136]	valid_0's l2: 2.88125e+06	valid_0's auc: 0.385436
[137]	valid_0's l2: 2.85585e+06	valid_0's auc: 0.386364
[138]	valid_0's l2: 2.8307e+06	valid_0's auc: 0.387291
[139]	valid_0's l2: 2.81583e+06	valid_0's auc: 0.386364
[140]	valid_0's l2: 2.79452e+06	valid_0's auc: 0.3859
[141]	valid_0's l2: 2.77323e+06	valid_0's auc: 0.387755
[142]	valid_0's l2: 2.74876e+06	valid_0's auc: 0.386827
[143]	valid_0's l2: 2.725e+06	valid_0's auc: 0.3859
[144]	valid_0's l2: 2.7014e+06	valid_0's auc: 0.384972
[145]	vali

In [ ]:
submission = pd.read_csv('/content/submission_auc_0.csv')

for i in range(1, FOLD):
    submission['Selling_Price'] +=  pd.read_csv(f'/content/submission_auc_{i}.csv')['Selling_Price']

submission['Selling_Price'] = submission['Selling_Price']/20
submission.to_csv('submission_ens4.csv')

In [ ]:
len(np.where(submission['Selling_Price']<0.0 , 0.0 , submission['Selling_Price'] ))

3430